In [33]:
import yaml
import os
import PIL
import numpy as np
import matplotlib.pyplot as plt
import nibabel as nib

def draw_image(image):
    return PIL.Image.fromarray((np.array(image) * 255).astype(np.uint8))


def read_nii_file(path):
    nifti = nib.load(path)
    data_array = nifti.get_data()
    affine_matrix = nifti.affine
    return data_array


In [34]:
import os
import numpy as np
import nibabel as nib
from tqdm import tqdm
import argparse
import monai
import torch
from torch.utils.tensorboard import SummaryWriter

def nifti_to_2d_slices(input_folder: str, output_folder: str, axis: int, filtered, resize):
    for fname in tqdm(sorted(os.listdir(input_folder))):

        if not fname.endswith("nii.gz"):
            continue

        n_file = os.path.join(input_folder, fname)
        nifti = nib.load(n_file)

        np_data = nifti.get_fdata()
        np_data = np_data.astype(np.float16)
        # np_data.shape (512, 512, 120)

        f_basename = fname.split(".")[0]

        for i in range(np_data.shape[axis]):
            slc = [slice(None)] * len(np_data.shape)
            
            slc[axis] = i
            image = np_data[slc]
            # image.shape (512, 512)

            if resize:
                tr = monai.transforms.Resize((resize, resize))
                image = tr(image[None])[0]

            if filtered:
                brain_mask = image > 0
                if brain_mask.sum() < 4000:
                    continue

            np.save(os.path.join(output_folder, f"{f_basename}_{i}.npy"), image)


input_dir = r"data\train"
output_dir = r"data\out"
axis = 2
do_filter = False
resize = 256
os.makedirs(output_dir, exist_ok=True)

nifti_to_2d_slices(input_dir, output_dir, axis, do_filter, resize)

  0%|          | 0/22 [00:00<?, ?it/s]C:\Users\HP\AppData\Local\Temp/ipykernel_15148/2005157495.py:29: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  image = np_data[slc]
100%|██████████| 22/22 [00:51<00:00,  2.33s/it]


In [41]:
res = read_nii_file(f'{input_dir}/crossmoda2021_ldn_1_ceT1.nii.gz')

#anw = np.array(res)
#print(res.dtype)
res = res.astype(float)
#writer = SummaryWriter("logs")
np_data = torch.tensor(res).unsqueeze(0)
print(np_data.shape)
#monai.visualize.img2tensorboard.plot_2d_or_3d_image(np_data,writer=writer,step=1)


C:\Users\HP\AppData\Local\Temp/ipykernel_15148/1041683615.py:14: DeprecationWarning: get_data() is deprecated in favor of get_fdata(), which has a more predictable return type. To obtain get_data() behavior going forward, use numpy.asanyarray(img.dataobj).

* deprecated from version: 3.0
* Will raise <class 'nibabel.deprecator.ExpiredDeprecationError'> as of version: 5.0
  data_array = nifti.get_data()


torch.Size([1, 512, 512, 120])


In [39]:


numpy_array = np.load('data/out\crossmoda2021_ldn_1_ceT1_1.npy')
numpy_array.shape

(256, 256)

In [42]:
from torch.utils.data.dataloader import DataLoader
from datasets import DatasetType
from datasets import DATASETS
from transforms import TRANSFORMS
with open('train_example.yaml', 'r') as stream:
    config = yaml.load(stream, Loader=yaml.FullLoader)




dataset_type = config['train_dataset']['dataset_type']
dataset_kwargs = config['train_dataset']['dataset_kwargs']
transform_kwargs = config['train_dataset']['transform_kwargs']




transform = TRANSFORMS[DatasetType[dataset_type]](**transform_kwargs)
dataset = DATASETS[DatasetType['numpy2d']](
    transform=transform,
    **dataset_kwargs
)

dl = DataLoader(dataset, batch_size=64, shuffle=True, drop_last=True, pin_memory=False)

next(iter(dl)).shape

0


c:\Users\HP\Desktop\AppDL\crossmoda-challenge\datasets.py:102: RuntimeWarning: overflow encountered in multiply
  image = PIL.Image.fromarray((np.array(image) * 255).astype(np.uint8))


torch.Size([64, 3, 64, 64])

In [44]:
dataset[3].shape

c:\Users\HP\Desktop\AppDL\crossmoda-challenge\datasets.py:102: RuntimeWarning: overflow encountered in multiply
  image = PIL.Image.fromarray((np.array(image) * 255).astype(np.uint8))


torch.Size([3, 64, 64])